# Polygons
---

In most simulations, such as Global Climate Models, Unstructured Grids are composed of nodes that are connected with edges to form faces that discretize the surface of a sphere (i.e. Earth).

For visualization, these faces can be geometrically represented as Polygons and shaded by a corresponding face-centered data variable.

## Setup

### Imports

In [ ]:
import uxarray as ux

file_dir = "../../meshfiles/"

### Grid with Face-Centered Data Variable

In [ ]:
grid_filename_mpas = file_dir + "oQU480.grid.nc"
data_filename_mpas = file_dir + "oQU480.data.nc"
uxds_mpas = ux.open_dataset(grid_filename_mpas, data_filename_mpas)

### Grid with Node-Centered Data Variable

In [ ]:
grid_filename_geoflow = file_dir + "geoflow.grid.nc"
data_filename_geoflow = file_dir + "geoflow.data.nc"
uxds_geoflow = ux.open_dataset(grid_filename_geoflow, data_filename_geoflow)

## Conversion Methods
UXarray represents Unstructured Grids through a set of coordinate and connectivity variables (i.e. `node_lon`, `node_lat`, `face_node_connectivity`, etc.). These variables can be manipulated to obtain our Polygons for visualuzation

### Representation as a `GeoDataFrame`

Polygons are stored in a SpatialPandas `GeoDataFrame`, which is the expected data structured by the HoloViz stack of packages for visualizing polygons.



A `Grid` can be converted into a `GeoDataFrame`, which will contain a single "geometry" column, which is a series of Polygons that represent each face.

In [ ]:
uxds_mpas.uxgrid.to_geodataframe()

A `UxDataArray` can also be converted into a `GeoDataFrame`. It will now have an additional column containing a 1D-slice of data variable.

It's important to note that to convert a `UxDataArray` into a `GeoDataFrame`, the dimension of the data variable must match the number of faces (a.k.a. mapped to faces) and there can not be any additional dimensions (i.e. time, level, etc.)

In [ ]:
uxds_mpas["bottomDepth"].to_geodataframe()

If a data variable is not face-centered, it can be manipulated to get it to map to faces. For node-centered data, as is the case with our Geoflow dataset, we can perform a nodal-average operation, which takes the average all the nodes that saddle a face and use that value to shade the polygon.

Here we can also see that we need to index the `time` and `meshLayers` dimensions to obtain our 1D slice of data.

In [ ]:
uxds_geoflow["v1"].nodal_average()[0][0].to_geodataframe()

## Vector Polygon Plots

In [ ]:
uxds_mpas["bottomDepth"].plot.polygons(width=900, height=400)

In [ ]:
uxds_geoflow["v1"].nodal_average()[0][0].plot.polygons(
    cmap="coolwarm", width=900, height=400
)

## Rasterized Polygon Plots

In [ ]:
uxds_mpas["bottomDepth"].plot.rasterize(method="polygon", width=900, height=400)

In [ ]:
uxds_geoflow["v1"].nodal_average()[0][0].plot.rasterize(
    method="polygon", cmap="coolwarm", width=900, height=400
)

## Handling Antimeridian Polygons

When attempting to visualize unstructured meshes that reside on a sphere, it's necessary to consider the behavior of geometric elements near the Antimeridian. Elements that exist on or cross the antimeridian need to be corrected to properly visualize them. UXarray uses the antimeridian package to split faces along the antimeridian. More details can be found in their [documentation](https://antimeridian.readthedocs.io/en/stable/).


![antimeridian example](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Earth_map_with_180th_meridian.jpg/640px-Earth_map_with_180th_meridian.jpg)


You can select whether to include or exclude these antimeridian polygons by using the `exclude_antimeridian` parameter. For larger, higher-resolution, grids, it's suggested to keep `exclude_antimeridian=True` due to the significantly faster performance. 

In [ ]:
(
    uxds_geoflow["v1"]
    .nodal_average()[0][0]
    .plot.polygons(
        exclude_antimeridian=False,
        cmap="coolwarm",
        title="With Antimeridian Faces",
        width=900,
        height=400,
    )
    + uxds_geoflow["v1"]
    .nodal_average()[0][0]
    .plot.polygons(
        exclude_antimeridian=True,
        cmap="coolwarm",
        title="Without Antimeridian Faces",
        width=900,
        height=400,
    )
).cols(1)